[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rsinghlab/pyaging/blob/main/tutorials/tutorial_dnam.ipynb) [![Open In nbviewer](https://img.shields.io/badge/View%20in-nbviewer-orange)](https://nbviewer.jupyter.org/github/rsinghlab/pyaging/blob/main/tutorials/tutorial_dnam.ipynb)

# Bulk DNA methylation

This tutorial is a brief guide for the implementation of an array of bulk DNA-methylation epigenetic clocks. In this notebook, we will demonstrate the breadth of epigenetic clock models available in *pyaging* by showing:

- Horvath's 2013 ElasticNet-based clock ([url](https://genomebiology.biomedcentral.com/articles/10.1186/gb-2013-14-10-r115));
  
- AltumAge, a highly accurate deep-learning based clock ([url](https://www.nature.com/articles/s41514-022-00085-y));
  
- PCGrimAge, a principal-component based version of the GrimAge clock ([url](https://www.nature.com/articles/s43587-022-00248-2));

We just need two packages for this tutorial.

In [1]:
import pandas as pd
import pyaging as pya

## Homo sapiens

### Download and load example data

Let's download the publicly avaiable dataset GSE139307 with Illumina's 450k array. The CpG coverage of the 450k array should be good enough for most clocks.

In [2]:
pya.data.download_example_data('GSE139307')

|-----> 🏗️ Starting download_example_data function
|-----------> Data found in pyaging_data/GSE139307.pkl
|-----> 🎉 Done! [0.0007s]


In [3]:
df = pd.read_pickle('pyaging_data/GSE139307.pkl')

In [4]:
df.head()

,dataset,tissue_type,age,gender,cg00000029,cg00000108,cg00000109,cg00000165,cg00000236,cg00000289,...,ch.X.93511680F,ch.X.938089F,ch.X.94051109R,ch.X.94260649R,ch.X.967194F,ch.X.97129969R,ch.X.97133160R,ch.X.97651759F,ch.X.97737721F,ch.X.98007042R
GSM4137709,GSE139307,sperm,84.0,M,0.084811,0.920696,0.856851,0.084567,0.838699,0.247273,...,0.061751,0.045942,0.037631,0.056455,0.249872,0.049022,0.085691,0.037435,0.077820,0.106234
GSM4137710,GSE139307,sperm,69.0,M,0.099626,0.919073,0.890024,0.115541,0.852584,0.198103,...,0.075077,0.041849,0.032573,0.089790,0.250245,0.079095,0.079756,0.046229,0.091256,0.120241
GSM4137711,GSE139307,sperm,69.0,M,0.117228,0.920276,0.894317,0.117127,0.839258,0.213410,...,0.068679,0.049515,0.058097,0.079919,0.299758,0.079305,0.089815,0.065364,0.086864,0.156005
GSM4137712,GSE139307,sperm,69.0,M,0.077096,0.910204,0.908400,0.073885,0.861615,0.163276,...,0.070091,0.033289,0.038836,0.108213,0.295428,0.050731,0.099943,0.047597,0.078480,0.107480
GSM4137713,GSE139307,sperm,67.0,M,0.063524,0.911608,0.884643,0.079877,0.864654,0.176169,...,0.082368,0.038411,0.048787,0.088631,0.316694,0.041873,0.079303,0.048823,0.089010,0.117903


For PCGrimAge, both age and sex are features. Therefore, to get the full prediction, let's convert the column `gender` into a column called `female`, with 1 being female and 0 being male. Let's just drop the other metadata for simplicity.

In [5]:
# needs only numerical data (doesn't work with strings)
df['female'] = (df['gender'] == 'F').astype(int)

### Convert data to AnnData object

AnnData objects are highly flexible and are thus our preferred method of organizing data for age prediction.

In [6]:
adata = pya.pp.df_to_adata(df, metadata_cols=['gender', 'tissue_type', 'dataset'], imputer_strategy='mean')

|-----> 🏗️ Starting df_to_adata function
|-----> ⚙️ Create anndata object started
|-----> ✅ Create anndata object finished [0.0678s]
|-----> ⚙️ Add metadata to anndata started
|-----------> Adding provided metadata to adata.obs
|-----> ✅ Add metadata to anndata finished [0.0008s]
|-----> ⚙️ Log data statistics started
|-----------> There are 37 observations
|-----------> There are 485514 features
|-----------> Total missing values: 526
|-----------> Percentage of missing values: 0.00%
|-----> ✅ Log data statistics finished [0.0245s]
|-----> ⚙️ Impute missing values started
|-----------> Imputing missing values using mean strategy
|-----> ✅ Impute missing values finished [0.2300s]
|-----> ⚙️ Add unstructured data to anndata started
|-----> ✅ Add unstructured data to anndata finished [0.0003s]
|-----> 🎉 Done! [0.4918s]


This is what the `adata` object looks like:

In [7]:
adata

AnnData object with n_obs × n_vars = 37 × 485514
    obs: 'gender', 'tissue_type', 'dataset'
    var: 'percent_na'
    uns: 'imputer_strategy', 'data_type'

### Predict age

We can either predict one clock at once or all at the same time. For convenience, let's simply input all three clocks of interest at once. The function is invariant to the capitalization of the clock name. 

In [8]:
adata = pya.pred.predict_age(adata, ['Horvath2013', 'AltumAge', 'PCGrimAge',])

|-----> 🏗️ Starting predict_age function
|-----> ⚙️ Set PyTorch device started
|-----------> Using device: cpu
|-----> ✅ Set PyTorch device finished [0.0010s]
|-----> Processing clock: Horvath2013
|-----------> ⚙️ Load clock started
|-----------------> Data found in pyaging_data/horvath2013.pt
|-----------> ✅ Load clock finished [0.0033s]
|-----------> ⚙️ Check features in adata started
|-----------------> All features are present in adata.var_names.
|-----------> ✅ Check features in adata finished [0.0016s]
|-----------> ⚙️ Filter features and extract data matrix started
|-----------> ✅ Filter features and extract data matrix finished [0.0059s]
|-----------> ⚙️ Convert numpy array to tensor started
|-----------> ✅ Convert numpy array to tensor finished [0.0006s]
|-----------> ⚙️ Initialize model started
|-----------> ✅ Initialize model finished [0.0021s]
|-----------> ⚙️ Predict ages with model started
|-----------> ✅ Predict ages with model finished [0.0008s]
|-----------> ⚙️ Convert

In [9]:
adata.obs.head()

,gender,tissue_type,dataset,horvath2013,altumage,pcgrimage
GSM4137709,M,sperm,GSE139307,33.624774,37.007030,68.520729
GSM4137710,M,sperm,GSE139307,28.829343,29.426702,58.770798
GSM4137711,M,sperm,GSE139307,28.316548,22.805408,57.613228
GSM4137712,M,sperm,GSE139307,24.850635,18.060072,59.368393
GSM4137713,M,sperm,GSE139307,25.942114,20.071892,59.199066


Having so much information printed can be overwhelming, particularly when running several clocks at once. In such cases, just set verbose to False.

In [10]:
pya.data.download_example_data('GSE139307', verbose=False)
df = pd.read_pickle('pyaging_data/GSE139307.pkl')
df['female'] = (df['gender'] == 'F').astype(int)
adata = pya.preprocess.df_to_adata(df, metadata_cols=['gender', 'tissue_type', 'dataset'], imputer_strategy='mean', verbose=False)
adata = pya.pred.predict_age(adata, ['Horvath2013', 'AltumAge', 'PCGrimAge',], verbose=False)

In [11]:
adata.obs.head()

,gender,tissue_type,dataset,horvath2013,altumage,pcgrimage
GSM4137709,M,sperm,GSE139307,33.624774,37.007030,68.520729
GSM4137710,M,sperm,GSE139307,28.829343,29.426702,58.770798
GSM4137711,M,sperm,GSE139307,28.316548,22.805408,57.613228
GSM4137712,M,sperm,GSE139307,24.850635,18.060072,59.368393
GSM4137713,M,sperm,GSE139307,25.942114,20.071892,59.199066


After age prediction, the clocks are added to `adata.obs`. Moreover, the percent of missing values for each clock and other metadata are included in `adata.uns`.

In [12]:
adata

AnnData object with n_obs × n_vars = 37 × 485514
    obs: 'gender', 'tissue_type', 'dataset', 'horvath2013', 'altumage', 'pcgrimage'
    var: 'percent_na'
    uns: 'imputer_strategy', 'data_type', 'horvath2013_metadata', 'altumage_metadata', 'pcgrimage_metadata'

### Get citation

The doi, citation, and some metadata are automatically added to the AnnData object under `adata.uns[CLOCKNAME_metadata]`.

In [13]:
adata.uns['horvath2013_metadata']

{'species': 'Homo sapiens',
 'data_type': 'methylation',
 'year': 2013,
 'citation': 'Horvath, Steve. "DNA methylation age of human tissues and cell types." Genome biology 14.10 (2013): 1-20.',
 'doi': 'https://doi.org/10.1186/gb-2013-14-10-r115'}

In [14]:
adata.uns['altumage_metadata']

{'species': 'Homo sapiens',
 'data_type': 'methylation',
 'year': 2022,
 'citation': 'de Lima Camillo, Lucas Paulo, Louis R. Lapierre, and Ritambhara Singh. "A pan-tissue DNA-methylation epigenetic clock based on deep learning." npj Aging 8.1 (2022): 4.',
 'doi': 'https://doi.org/10.1038/s41514-022-00085-y'}

In [15]:
adata.uns['pcgrimage_metadata']

{'species': 'Homo sapiens',
 'data_type': 'methylation',
 'year': 2022,
 'citation': 'Higgins-Chen, Albert T., et al. "A computational solution for bolstering reliability of epigenetic clocks: Implications for clinical trials and longitudinal tracking." Nature aging 2.7 (2022): 644-661.',
 'doi': 'https://doi.org/10.1038/s43587-022-00248-2'}

## Mammalian

### Download and load example data

Let's download the publicly avaiable dataset GSE223748 with Illumina's Mammalian Methylation array. The CpG coverage of the this array (~37k) spans highly conserved CpG sequences. Let's download a subset of that data.

In [16]:
pya.data.download_example_data('GSE223748')

|-----> 🏗️ Starting download_example_data function
|-----------> Data found in pyaging_data/GSE223748_subset.pkl
|-----> 🎉 Done! [0.0015s]


In [17]:
df = pd.read_pickle('pyaging_data/GSE223748_subset.pkl')

In [18]:
df.head()

,cg00000165,cg00001209,cg00001364,cg00001582,cg00002920,cg00003994,cg00004555,cg00005112,cg00005271,cg00006213,...,rs7746156_II_F_C_37550,rs798149_II_F_C_37528,rs845016_II_F_C_37529,rs877309_II_F_C_37552,rs9292570_I_F_C_37499,rs9363764_II_F_C_37541,rs939290_II_F_C_37535,rs951295_I_F_C_37507,rs966367_II_F_C_37551,rs9839873_II_F_C_37532
204509080002_R01C02,0.094879,0.916154,0.890314,0.053583,0.490381,0.034852,0.159705,0.763959,0.973245,0.928975,...,0.488592,0.491361,0.480024,0.500000,0.484252,0.489448,0.505585,0.505335,0.485003,0.510081
202897220142_R04C02,0.497077,0.441263,0.915314,0.047339,0.651029,0.037774,0.082634,0.415800,0.702857,0.821715,...,0.508102,0.500299,0.507261,0.490684,0.499673,0.497256,0.564106,0.482151,0.486667,0.505236
204529320092_R01C02,0.321141,0.834158,0.881194,0.056124,0.688350,0.030225,0.086776,0.777588,0.974587,0.923934,...,0.520404,0.509568,0.507549,0.501659,0.492823,0.487243,0.516018,0.471244,0.491066,0.491759
202794570004_R02C01,0.495226,0.924121,0.915812,0.050866,0.688335,0.032344,0.113318,0.872094,0.969189,0.917076,...,0.499314,0.516132,0.487009,0.487146,0.469119,0.495125,0.548238,0.512283,0.514257,0.492520
203531420070_R05C02,0.183954,0.934332,0.924153,0.055032,0.717495,0.037108,0.098632,0.859614,0.973422,0.963446,...,0.501432,0.509412,0.485055,0.497272,0.480637,0.467502,0.494246,0.500924,0.531334,0.503709


### Convert data to AnnData object

AnnData objects are highly flexible and are thus our preferred method of organizing data for age prediction.

In [19]:
adata = pya.pp.df_to_adata(df, imputer_strategy='mean')

|-----> 🏗️ Starting df_to_adata function
|-----> ⚙️ Create anndata object started
|-----> ✅ Create anndata object finished [0.0014s]
|-----> ⚙️ Add metadata to anndata started
|-----------? No metadata provided. Leaving adata.obs empty
|-----> ⚠️ Add metadata to anndata finished [0.0007s]
|-----> ⚙️ Log data statistics started
|-----------> There are 100 observations
|-----------> There are 37554 features
|-----------> Total missing values: 0
|-----------> Percentage of missing values: 0.00%
|-----> ✅ Log data statistics finished [0.0035s]
|-----> ⚙️ Impute missing values started
|-----------> No missing values found. No imputation necessary
|-----> ✅ Impute missing values finished [0.0058s]
|-----> ⚙️ Add unstructured data to anndata started
|-----> ✅ Add unstructured data to anndata finished [0.0006s]
|-----> 🎉 Done! [0.0157s]


This is what the `adata` object looks like:

In [20]:
adata

AnnData object with n_obs × n_vars = 100 × 37554
    var: 'percent_na'
    uns: 'imputer_strategy', 'data_type'

### Predict age

We can either predict one clock at once or all at the same time. For convenience, let's simply input all four available mammalian clocks at once. The function is invariant to the capitalization of the clock name.

In [21]:
adata = pya.pred.predict_age(adata, ['Mammalian1', 'Mammalian2', 'Mammalian3', 'MammalianLifespan',])

|-----> 🏗️ Starting predict_age function
|-----> ⚙️ Set PyTorch device started
|-----------> Using device: cpu
|-----> ✅ Set PyTorch device finished [0.0008s]
|-----> Processing clock: Mammalian1
|-----------> ⚙️ Load clock started
|-----------------> Data found in pyaging_data/mammalian1.pt
|-----------> ✅ Load clock finished [0.0020s]
|-----------> ⚙️ Check features in adata started
|-----------------> All features are present in adata.var_names.
|-----------> ✅ Check features in adata finished [0.0008s]
|-----------> ⚙️ Filter features and extract data matrix started
|-----------> ✅ Filter features and extract data matrix finished [0.0016s]
|-----------> ⚙️ Convert numpy array to tensor started
|-----------> ✅ Convert numpy array to tensor finished [0.0005s]
|-----------> ⚙️ Initialize model started
|-----------> ✅ Initialize model finished [0.0008s]
|-----------> ⚙️ Predict ages with model started
|-----------> ✅ Predict ages with model finished [0.0004s]
|-----------> ⚙️ Convert t

Of note, Mammalian1 is age in units of years, Mammalian2 is the relative age (0 to 1), and Mammalian 3 is the log-linear transformed version of age (in order to get the age in years, one needs to know the "adult_age" of the species tested for the inverse transformation).

In [22]:
adata.obs.head()

,mammalian1,mammalian2,mammalian3,mammalianlifespan
204509080002_R01C02,26.372459,0.313471,0.250649,93.886052
202897220142_R04C02,1.176587,0.356768,1.384633,6.999166
204529320092_R01C02,18.776440,0.253814,-0.202201,73.335157
202794570004_R02C01,0.890974,0.320042,1.136358,5.332617
203531420070_R05C02,10.371320,0.142493,-0.559288,68.409359


Having so much information printed can be overwhelming, particularly when running several clocks at once. In such cases, just set verbose to False.

In [23]:
pya.data.download_example_data('GSE223748', verbose=False)
df = pd.read_pickle('pyaging_data/GSE223748_subset.pkl')
adata = pya.preprocess.df_to_adata(df, imputer_strategy='mean', verbose=False)
adata = pya.pred.predict_age(adata, ['Mammalian1', 'Mammalian2', 'Mammalian3', 'MammalianLifespan',], verbose=False)

In [24]:
adata.obs.head()

,mammalian1,mammalian2,mammalian3,mammalianlifespan
204509080002_R01C02,26.372459,0.313471,0.250649,93.886052
202897220142_R04C02,1.176587,0.356768,1.384633,6.999166
204529320092_R01C02,18.776440,0.253814,-0.202201,73.335157
202794570004_R02C01,0.890974,0.320042,1.136358,5.332617
203531420070_R05C02,10.371320,0.142493,-0.559288,68.409359


After age prediction, the clocks are added to `adata.obs`. Moreover, the percent of missing values for each clock and other metadata are included in `adata.uns`.

In [25]:
adata

AnnData object with n_obs × n_vars = 100 × 37554
    obs: 'mammalian1', 'mammalian2', 'mammalian3', 'mammalianlifespan'
    var: 'percent_na'
    uns: 'imputer_strategy', 'data_type', 'mammalian1_metadata', 'mammalian2_metadata', 'mammalian3_metadata', 'mammalianlifespan_metadata'

### Get citation

The doi, citation, and some metadata are automatically added to the AnnData object under `adata.uns[CLOCKNAME_metadata]`.

In [26]:
adata.uns['mammalian1_metadata']

{'species': 'multi',
 'data_type': 'methylation',
 'year': 2023,
 'citation': 'Lu, A. T., et al. "Universal DNA methylation age across mammalian tissues." Nature aging 3.9 (2023): 1144-1166.',
 'doi': 'https://doi.org/10.1038/s43587-023-00462-6'}